# Understanding Recurrent Neural Networks

Sources:

Andrej Karpathy blog: The Unreasonable Effectiveness of Recurrent Neural Networks http://karpathy.github.io/2015/05/21/rnn-effectiveness/

Stanford cs231n (spring 2017) lecture 10: Recurrent Neural Networks https://www.youtube.com/watch?v=6niqTuYFZLQ

Visualizing and Understanding Recurrent Networks https://arxiv.org/abs/1506.02078

## Context:

Neural networks like CNNs typically require some fixed-size input, and produce a fixed-size output (see one-to-one in figure below).

RNNs can operate on every item of a sequence; so the length of that sequence can very in size. The output can also vary in size.

- One to many: eg, given an input image, produce a sequence of words that describes it (image captioning)
- Many to one: eg, given an input sequence of words, produce a single label for the text (sentiment classification)
- Many to many: eg, given a sequence of English words, produce a sequence of French words (machine translation); or given a sequence of video frames, produce a sentence describing the scene.

You can also iterate over fixed-sized inputs on an RNN.

![img](./img/rnn-in-out-size.jpg)

## How does a RNN work?

Like a static variable in a class that gets updated every time some method is called, the hidden state in a RNN is updated by a new input. The updated hidden state is fed back into the model the next time it reads a new input.

![img](./img/rnn-recurrence-formula.png)

### Calculating new state and _recurrence_:

For example: for every word in a sentence, run the word through the RNN function (the input word can be a one-hot encoded vector):

$$h_t = \tanh ( W_{hh} h_{t-1} + W_{xh} x_t )$$

At the first step ($t = 1$), the function takes the first word $x_1$, and a hidden state $h$ as inputs. Since the hidden state is initialized to a vector of zeroes, the first hidden state is essentially the tanh of the first word (tanh is applied element-wise; it squashes each value to betwenn -1 and 1):

$$h_1 = \tanh ( W_{xh} x_1 )$$

![img](./img/rnn-step1.png)

The second word $x_2$ together with previous step's hidden state $h_1$ are fed into the same function to produce a new hidden state $h_2$:

$$h_2 = \tanh ( W_{hh} h_1 + W_{xh} x_2 )$$

![img](./img/rnn-step2.png)

This process is repeated until the end of the sentence.

![img](./img/rnn-step3.png)

Both $x_t$ and $h_t$ have their own set of weights $W_{xh}$ and $W_{hh}$ (as a fully connected layer) that remain unchanged during the forward pass.

![img](./img/rnn-step3-weights.png)

If you want to produce an output $y_t$ at each step, you can introduce another set of weights $W_{hy}$ for the calculation:

$$y_t = W_{hy} \times h_t $$

![img](./img/rnn-step3-outputs.png)

In code, it looks like this:

In [ ]:
class RNN:
    '''A single recurrent cell'''
    def step(self, x):
        '''Update the hidden state'''
        self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
        # Optional: compute the output vector
        y = np.dot(self.W_hy, self.h)
        return y

rnn = RNN()

Reminder: `np.dot(a, b)` is the [dot-product](https://en.wikipedia.org/wiki/Dot_product#Algebraic_definition) of 2 vectors (__not__ element-wise!).

`np.tanh` is applied element-wise.

Notice that there are __three sets of weights__! $W_{hh}$ (`self.W_hh`) for the hidden state; $W_{xh}$ (`self.W_xh`) for the input; and $W_{hy}$ (`self.W_hy`) for the output.

The hidden state `self.h` is initialized to a vector of zeros.

`np.tanh` function implements a non-linearity that squashes the activations to the range `[-1, 1]`.

For each step in a sequence, you'd run the line below to update the hidden state:

In [ ]:
rnn.step(x) # x is an input vector

Of course, you can daisy-chain (stack) RNN models so that the output of one cell becomes the input of a downstream cell:

In [ ]:
y1 = rnn.step(x)
y2 = rnn2.step(y1)

For each prediction, there's the accompanying loss (usually softmax loss):

![img](./img/rnn-step3-outputs-loss.png)

The final loss is the sum of all individual loss at each step:

![img](./img/rnn-step4-final-loss.png)

On the backwards pass, the loss gradient flows through each time-step, and each step will compute the local gradient for weights $W_{hh}$, $W_{xh}$, and $W_{hy}$. Which are then summed for the final gradient for the weights $W$.

In a __many-to-one__ model, the last hidden state $h_T$ at the end of the sentence can be considered a summary of the input sentence.

Whereas for a __one-to-many__ model, the input is an initializer for step 1 of the hidden state.

A __sequence-to-sequence__ model (eg, neural machine translation) is basically a many-to-one model placed before a one-to-many model.

It operates in 2 stages:
1. the model upfront __encodes__ the input sequence of words to a single summary vector. That vector is the hidden state of the last step of the model.
1. the model downstream __decodes__ that vector into a sequence of words in another language.

![img](./img/rnn-seq2seq.png)

### RNN example


Let's say that we have a character-level model that predicts what the next letter should be given an input letter. Assume the vocabulary consists of only 4 letters: h, e, l, o; and we're training the model to predict the word 'hello'.

The characters are first converted to a 4-element, one-hot vector:

![img](./img/rnn-char-seq-inputs.jpg)

#### Training

The input vector multiplies by the input weight matrix $W_{xh}$ to get the first hidden state. That hidden state is then multiplied by an output weight matrix $W_{hy}$ to produce a list of scores for each letter:

![img](./img/rnn-char-seq.jpg)

For the first letter 'h', the correct next letter should be 'e', but it gave 'o' a higher score. In this case, we'd use a softmax loss to quantify (a scalar) how wrong the prediction is, and that loss's gradient will be fed back into the cell during the backwards pass. This process repeats during training.

#### Testing

At test time, each input's score is converted to a probability distribution by a softmax function. A prediction is then __sampled__ from that distribution. That prediction is then fed back for the next time-step:

![img](./img/rnn-eg-1.png)

And this process repeats:

![img](./img/rnn-eg-2.png)

#### Questions

Why __sample__ from the distribution? Why not just take the character with the highest score (argmax)?

- Sometimes you do take the argmax. The advantage with sampling is that you get variety in your outputs so that you don't always end up with the same output given the same input. eg, the same image can be captioned in a few different ways by the model.

During test time, when the first prediction is made, can you feed the softmax score into the next round (instead of using the one-hot vector)?

- No, because the softmax scores look very different from what the model saw during training. This can cause bad outputs.
- The other problem is that the using a dense vector as an input can be computationally expensive. If your vocabulary size is 10,000, then the input vector becomes a dense softmax vector of 10,000 numbers.

### Backpropagation through time

During the forward pass, you're stepping through time to compute the loss. During the backward pass, you're stepping backwards through time to compute the gradient.

What if the input sequence is very long? Like Wikipedia-sized long? You can't just run through all wikipedia text forward and backward to produce one gradient update; you'll run out of memory and never converge because it's so slow.

Just as you'd make a gradient update after every few images for a CNN, you can do the same thing (mini-batch) to gradient updates in a RNN:  you run a gradient update once every few words (say, 100 words). This is known as a __truncated backpropagation through time__.

![img](./img/rnn-truncated-bptt.png)

### The hidden state visualized

What exactly does it 'remember'?

[This paper](https://arxiv.org/abs/1506.02078) selects one number from the hidden state vector, and see which characters cause a spike in activation when an input sequence is fed into the model.

Most elements from the hidden state vector aren't easily interpretable:

![img](./img/rnn-interpret1.png)

But some elements activate in a more interpretable way:

Quote detection:

![img](./img/rnn-interpret2.png)

Line break:

![img](./img/rnn-interpret3.png)

`if` statement conditions:

![img](./img/rnn-interpret4.png)

The take-away is that even though the model was trained to predict the next character, it also learned useful structural rules of the input data.

### Image captioning

A CNN distills the image down to a summary vector $v$, it becomes another input to the hidden state formula. $v$ also has its own weights $W_{ih}$.

![img](./img/rnn-img-caption.png)

The initial input is a special `<START>` token.

Previously, the hidden state was calculated like this:

$$h_t = \tanh ( W_{hh} h_{t-1} + W_{xh} x_t )$$

Now we have an image vector $v$ and its weights $W_{ih}$ to account for:

$$h_t = \tanh ( W_{hh} h_{t-1} + W_{xh} x_t + W_{ih} v)$$

__It's important to note that the image vector is not used as the input $x_t$__!

For training, the labels must have special tokens marking the `<START>` and `<END>` of the sentence. This tells the network to stop generating words whenever it has sampled an `<END>` token.



![img](./img/rnn-img-caption2.png)

![img](./img/rnn-img-caption3.png)

Example results:

![img](./img/rnn-img-caption-eg.png)

Bad results:

![img](./img/rnn-img-caption-eg2.png)

### Image captioning with attention

instead of outputting a single summary vector, it outputs a grid of vectors. imagine the image is divided into grids of 9 squares. then the output matrix will have 9 vectors, each corresponding to its own location in the grid.

besides sampling the output vocab, the model also produces a distribution of image locations that it wants to look. This distribution can be seen as the attention of the model (ie, where it is looking at).



## LSTM